In [ ]:
import sys

assert sys.version_info >=(3,8), "This project requires Python 3.8+"

In [ ]:
from packaging import version
import torch


assert version.parse(torch.__version__) >= version.parse("2.1.2"), "This project requires pytorch 2.1.1 or above!"

In [ ]:
import os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [ ]:
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm

torch.random.manual_seed(42)

**DATASETS & DATALOADERS**

* Code for processing data samples can get messy and hard to maintain; we ideally want our dataset code to be decoupled from our model training code for better readability and modularity. PyTorch provides two data primitives: torch.utils.data.DataLoader and torch.utils.data.Dataset that allow you to use pre-loaded datasets as well as your own data. Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset to enable easy access to the samples.

* PyTorch domain libraries provide a number of pre-loaded datasets (such as FashionMNIST) that subclass torch.utils.data.Dataset and implement functions specific to the particular data. They can be used to prototype and benchmark your model. You can find them here: Image Datasets, Text Datasets, and Audio Datasets



In [ ]:
from torch.utils.data import random_split

train_dataset = datasets.FashionMNIST(root="./data", train=True, download=True, transform=ToTensor())
test_ds = datasets.FashionMNIST(root="./data", train=False, download=True, transform=ToTensor())

train_ds, val_ds = random_split(train_dataset, [50000, 10000])


In [ ]:
print("train+val size: ", train_dataset.data.shape)
print("test size: ", test_ds.data.shape)

In [ ]:
## Hyperparameters
input_size = [*train_dataset.data.shape[1:]]
number_epochs = 10
batch_size = 32
learning_rate = 0.01

In [ ]:
train_dloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dloader = DataLoader(val_ds, batch_size=batch_size, shuffle=True)
test_dloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

In [ ]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in tqdm(range(1, cols * rows + 1)):
    sample_idx = torch.randint(len(train_dataset), size=(1,)).item()
    img, label = train_dataset[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

In [ ]:
class NN(nn.Module):
    def __init__(self, in_feature: int, output_feature: int,   *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_feature, 300)
        self.bn1 = nn.BatchNorm1d(300)
        self.fc2 = nn.Linear(300, 100)
        self.bn2 = nn.BatchNorm1d(100)
        self.fc3 = nn.Linear(100, output_feature)



    def forward(self, x):
        x = self.flatten(x)
        output = self.bn1(self.fc1(x))
        output = nn.GELU()(output)
        output = self.bn2(self.fc2(output))
        output = nn.GELU()(output)
        return self.fc3(output)

In [ ]:
# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import shutil

saving_path = 'saved_models/fashion_mnist'


# model = nn.Sequential(nn.Flatten(),
#     nn.Linear(784, 300),
#     nn.ReLU(),
#     nn.Dropout(0.2),
#     nn.Linear(300, 100),
#     nn.ReLU(),
#     nn.Dropout(0.2),
#     nn.Linear(100, 10))

model = NN(784, 10)

# model description
print("-" * 20)
print("model: ", model)
model_total_params = sum(p.numel() for p in model.parameters())
model_total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("model total parameters: ", model_total_params)
print("model total trainable parameters: ", model_total_trainable_params)
print("-" * 20)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter(f'runs/fashion_trainer_{timestamp}')

epoch_number = 0
best_vloss = 1_000_000.
train_losses, val_losses = [], []


# Train & validate Network
for epoch in range(number_epochs):
    print(f'EPOCH {epoch + 1}:')
    running_loss = 0.
    last_loss = 0.
    model.train(True)
    for batch_idx, (data, targets) in enumerate(tqdm(train_dloader)):

        data = data.to(device=device)
        targets = targets.to(device=device)

        # Make predictions for this batch (forward)
        outputs = model(data)

        # Compute the loss and its gradients
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()

        # Adjust learning weights
        optimizer.step()


        running_loss += loss.item()

        if batch_idx % 500 == 499:
            last_loss = running_loss / 500 # loss per batch
            print(f'batch {batch_idx + 1} loss: {last_loss}')
            tb_x = epoch_number * len(train_dloader) + batch_idx + 1
            writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    running_vloss = 0.0
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, (vinputs, vlabels) in enumerate(val_dloader):
            voutputs = model(vinputs)
            vloss = criterion(voutputs, vlabels)
            running_vloss += vloss
    avg_vloss = running_vloss / (i + 1)
    print(f'LOSS train {last_loss} valid {avg_vloss}')
    
    train_losses.append(last_loss)
    val_losses.append(avg_vloss)

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : last_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        if os.path.exists(saving_path):
            shutil.rmtree(saving_path)
        os.makedirs(saving_path)
        model_path = f'{saving_path}/model_{timestamp}_{epoch_number}.pth'
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

In [ ]:
%load_ext tensorboard

%tensorboard --logdir=runs

In [ ]:
plt.plot(train_losses, label = "Training loss")
plt.plot(val_losses, label = "Validation loss")
plt.legend(frameon = False)

In [ ]:
import glob

if len(glob.glob(f"{saving_path}/*.pth")) > 0:
    saved_model_params = glob.glob(f"{saving_path}/*.pth")[0]

loaded_model = NN(784, 10)
# loaded_model = nn.Sequential(nn.Flatten(),
#     nn.Linear(784, 300),
#     nn.ReLU(),
#     nn.Linear(300, 100),
#     nn.ReLU(),
#     nn.Linear(100, 10))
loaded_model.load_state_dict(torch.load(saved_model_params))

In [ ]:
from bootcamp_libs.metrics.accuracy import check_accuracy

print("train accuracy: ", check_accuracy(train_dloader, loaded_model).item())
print("val accuracy: ", check_accuracy(val_dloader, loaded_model).item())
print("test accuracy: ", check_accuracy(test_dloader, loaded_model).item())